# Lot compounding

This notebook looks at methods on compounding lots; futures and options contracts that could be traded only in specific lots. Since these cannot be compounded in a fine grained manner, let us start with some simple methods.

Assumptions
------------

 1. Profit and loss is expressed in points per contract
 2. Capital required per contract is fairly stable over time. This is important in case you are **selling options**.
 3. Commissions and slippage is included in the profit and loss
 
 

## Simple Method

In the simple method, we allocate capital per lot based on capital required per lot and add maximum expected loss for a single lot. Maximum expected loss is the maximum loss per trade multiplied by the number of expected consecutive losing trade. We assume we could **estimate maximum loss per trade reasonably well.**


Example
-------
    Lot size = 100
    Capital required per lot = 50000
    Maximum loss in points per trade = 100
    Expected consecutive losses = 5
    Expected maximum loss = 100*5*100 = 50000
    Total capital to be allocated = 50000+50000 = 100000
    

So, lots would be increased if we earn necessary capital per lot and we would keep on trading till we exhaust our buffer. In this case, we would increase the lot size if we earn 50000 and keep on trading till our total capital falls to 50000.

> Number of lots to trade = **int((total capital - expected maximum loss)/capital per lot)**


In [19]:
import pandas as pd
import numpy as np
from typing import Union, List, Tuple

In [60]:
def lot_compounding(pnl:Union[pd.Series, List, Tuple], initial_capital:float=100000,
                   capital_per_lot:float=50000, buffer:float=50000, lot_size:float=100)->pd.DataFrame:
    """
    Compounding based on loss
    pnl
        profit or loss in points per contract
        points should not be multiplied by lot size
    initial_capital
        initial capital invested includes both capital per lot and buffer
    capital_per_lot
        capital required per lot
    buffer
        buffer for drawdown for a single lot
    lot_size
        lot size
    """
    capital = initial_capital
    capital_arr = []
    lots_arr = []
    profit_arr = []
    for p in pnl:
        lots = int((capital-buffer)/capital_per_lot)
        if lots == 0:
            if capital > capital_per_lot:
                lots = 1
        if lots > 0:
            profit = p*lot_size*lots
            capital += profit
        else:
            profit = 0
        capital_arr.append(capital)
        lots_arr.append(lots)
        profit_arr.append(profit)
        index = [pnl.index if type(pnl)==pd.Series else np.arange(len(pnl))]
    return pd.DataFrame({
        'lots': lots_arr,
        'capital': capital_arr,
        'profit': profit_arr,
    }, index=index)

            

## Let us try a few simple cases

In [61]:
# All profit

pnl = [200] * 6
lot_compounding(pnl)

,lots,capital,profit
0,1,120000,20000
1,1,140000,20000
2,1,160000,20000
3,2,200000,40000
4,3,260000,60000
5,4,340000,80000


In [62]:
# Max consecutive losses
pnl = [-100]*6 
lot_compounding(pnl)

,lots,capital,profit
0,1,90000,-10000
1,1,80000,-10000
2,1,70000,-10000
3,1,60000,-10000
4,1,50000,-10000
5,0,50000,0


In [87]:
# Big profits followed by consecutive losses
# Think 470 as an extreme case of error. You must never have it in your strategy
pnl = [1000]*3 + [-470] + [-160]*5 + [150]
print(pnl)
lot_compounding(pnl)

[1000, 1000, 1000, -470, -160, -160, -160, -160, -160, 150]


,lots,capital,profit
0,1,200000,100000
1,3,500000,300000
2,9,1400000,900000
3,27,131000,-1269000
4,1,115000,-16000
5,1,99000,-16000
6,1,83000,-16000
7,1,67000,-16000
8,1,51000,-16000
9,1,66000,15000
